In [2]:
import os

os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_HOME'] = '/usr/local/cuda-7.5'

In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers import merge,Input 
from keras.engine import Model
from keras.optimizers import SGD
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import keras.backend as K
import json
import time
nb_classes=85

/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# Reading the train and test meta-data files
import pandas as pd
train = pd.read_csv('DL3 Dataset/train.csv')
test = pd.read_csv('DL3 Dataset/test.csv')

train.head()

# Shape of training and test datasets
print ('Training dataset consists of {} images with {} attributes'.format(train.shape[0], train.shape[1]-1))
# Shape of training and test datasets
print ('Testing dataset consists of {} images.'.format(test.shape[0]))

Training dataset consists of 12600 images with 85 attributes
Testing dataset consists of 5400 images.


In [5]:
TRAIN_PATH = 'DL3 Dataset/train/'
TEST_PATH = 'DL3 Dataset/test/'
img_path = TRAIN_PATH+str(train.Image_name[0])
from PIL import Image
import cv2


# Extracting label columns
label_cols = list(set(train.columns) - set(['Image_name']))
label_cols.sort()


# Extracting labels corresponding to image at the zeroth index of the training dataset.
labels = train.iloc[0][2:].index[train.iloc[0][2:] == 1]



In [6]:
from tqdm import tqdm
def read_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128,128))
    return img

In [7]:
train_img = []
for img_path in tqdm(train.Image_name.values):
    train_img.append(read_img(TRAIN_PATH + img_path))

100%|██████████| 12600/12600 [06:23<00:00, 32.88it/s]


In [8]:
#reorder dimensions for tensorflow
X_train = np.array(train_img, np.float32) / 255.
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
del train_img

('x_train shape:', (12600, 128, 128, 3))
(12600, 'train samples')


import gc
del train_img
gc.collect()

mean_img = X_train.mean(axis=0)
std_dev = X_train.std(axis = 0)
x_train = (X_train - mean_img)/ std_dev

del X_train
gc.collect()

In [9]:
y_train = train[label_cols].values
import gc
gc.collect()

7

In [10]:


Xtrain, Xvalid, ytrain, yvalid = train_test_split(X_train, y_train, test_size=0.2, random_state=47)

del X_train
gc.collect()
#del train_img

0

# test set

In [11]:
test_img = []
for img_path in tqdm(test.Image_name.values):
    test_img.append(read_img(TEST_PATH + img_path))

100%|██████████| 5400/5400 [02:30<00:00, 35.97it/s]


In [12]:
X_test = np.array(test_img, np.float32) / 255.
del test_img
gc.collect()

0

mean_img = X_test.mean(axis=0)
std_dev = X_test.std(axis = 0)
Xtest = (X_test - mean_img)/ std_dev

del X_test
gc.collect()

# densenet

In [14]:
from keras.optimizers import SGD
from keras.layers import Input, merge, ZeroPadding2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model


from custom_layers.scale_layer import Scale

In [15]:
def densenet169_model(img_rows, img_cols, color_type=1, nb_dense_block=4, growth_rate=32, nb_filter=64, reduction=0.5, dropout_rate=0.0, weight_decay=1e-4, num_classes=None):
    '''
    DenseNet 169 Model for Keras

    Model Schema is based on 
    https://github.com/flyyufelix/DenseNet-Keras

    ImageNet Pretrained Weights 
    Theano: https://drive.google.com/open?id=0Byy2AcGyEVxfN0d3T1F1MXg0NlU
    TensorFlow: https://drive.google.com/open?id=0Byy2AcGyEVxfSEc5UC1ROUFJdmM

        A Keras model instance.
    '''
    eps = 1.1e-5

    # compute compression factor
    compression = 1.0 - reduction

    # Handle Dimension Ordering for different backends
    global concat_axis
    if K.image_dim_ordering() == 'tf':
      concat_axis = 3
      img_input = Input(shape=(128,128, 3), name='data')
    else:
      concat_axis = 1
      img_input = Input(shape=(3, 128,128), name='data')

    # From architecture for ImageNet (Table 1 in the paper)
    nb_filter = 64
    nb_layers = [6,12,32,32] # For DenseNet-169

    # Initial convolution
    x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)
    x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=concat_axis, name='conv1_bn')(x)
    x = Scale(axis=concat_axis, name='conv1_scale')(x)
    x = Activation('relu', name='relu1')(x)
    x = ZeroPadding2D((1, 1), name='pool1_zeropadding')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        stage = block_idx+2
        x, nb_filter = dense_block(x, stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)

        # Add transition_block
        x = transition_block(x, stage, nb_filter, compression=compression, dropout_rate=dropout_rate, weight_decay=weight_decay)
        nb_filter = int(nb_filter * compression)

    final_stage = stage + 1
    x, nb_filter = dense_block(x, final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)

    x = BatchNormalization(epsilon=eps, axis=concat_axis, name='conv'+str(final_stage)+'_blk_bn')(x)
    x = Scale(axis=concat_axis, name='conv'+str(final_stage)+'_blk_scale')(x)
    x = Activation('relu', name='relu'+str(final_stage)+'_blk')(x)

    x_fc = GlobalAveragePooling2D(name='pool'+str(final_stage))(x)
    x_fc = Dense(1000, name='fc6')(x_fc)
    x_fc = Activation('softmax', name='prob')(x_fc)

    model = Model(img_input, x_fc, name='densenet')

    if K.image_dim_ordering() == 'th':
      # Use pre-trained weights for Theano backend
      weights_path = 'densenet169_weights_th.h5'
    else:
      # Use pre-trained weights for Tensorflow backend
      weights_path = 'densenet169_weights_tf.h5'

    model.load_weights(weights_path, by_name=True)

    # Truncate and replace softmax layer for transfer learning
    # Cannot use model.layers.pop() since model is not of Sequential() type
    # The method below works since pre-trained weights are stored in layers but not in the model
    x_newfc = GlobalAveragePooling2D(name='pool'+str(final_stage))(x)
    x_newfc = Dense(num_classes, name='fc6')(x_newfc)
    x_newfc = Activation('sigmoid', name='prob')(x_newfc)

    model = Model(img_input, x_newfc)

    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

    return model


def conv_block(x, stage, branch, nb_filter, dropout_rate=None, weight_decay=1e-4):
    '''Apply BatchNorm, Relu, bottleneck 1x1 Conv2D, 3x3 Conv2D, and option dropout
        # Arguments
            x: input tensor 
            stage: index for dense block
            branch: layer index within each dense block
            nb_filter: number of filters
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''
    eps = 1.1e-5
    conv_name_base = 'conv' + str(stage) + '_' + str(branch)
    relu_name_base = 'relu' + str(stage) + '_' + str(branch)

    # 1x1 Convolution (Bottleneck layer)
    inter_channel = nb_filter * 4  
    x = BatchNormalization(epsilon=eps, axis=concat_axis, name=conv_name_base+'_x1_bn')(x)
    x = Scale(axis=concat_axis, name=conv_name_base+'_x1_scale')(x)
    x = Activation('relu', name=relu_name_base+'_x1')(x)
    x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    # 3x3 Convolution
    x = BatchNormalization(epsilon=eps, axis=concat_axis, name=conv_name_base+'_x2_bn')(x)
    x = Scale(axis=concat_axis, name=conv_name_base+'_x2_scale')(x)
    x = Activation('relu', name=relu_name_base+'_x2')(x)
    x = ZeroPadding2D((1, 1), name=conv_name_base+'_x2_zeropadding')(x)
    x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    return x


def transition_block(x, stage, nb_filter, compression=1.0, dropout_rate=None, weight_decay=1E-4):
    ''' Apply BatchNorm, 1x1 Convolution, averagePooling, optional compression, dropout 
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_filter: number of filters
            compression: calculated as 1 - reduction. Reduces the number of feature maps in the transition block.
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''

    eps = 1.1e-5
    conv_name_base = 'conv' + str(stage) + '_blk'
    relu_name_base = 'relu' + str(stage) + '_blk'
    pool_name_base = 'pool' + str(stage) 

    x = BatchNormalization(epsilon=eps, axis=concat_axis, name=conv_name_base+'_bn')(x)
    x = Scale(axis=concat_axis, name=conv_name_base+'_scale')(x)
    x = Activation('relu', name=relu_name_base)(x)
    x = Convolution2D(int(nb_filter * compression), 1, 1, name=conv_name_base, bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    x = AveragePooling2D((2, 2), strides=(2, 2), name=pool_name_base)(x)

    return x


def dense_block(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1e-4, grow_nb_filters=True):
    ''' Build a dense_block where the output of each conv_block is fed to subsequent ones
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_layers: the number of layers of conv_block to append to the model.
            nb_filter: number of filters
            growth_rate: growth rate
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            grow_nb_filters: flag to decide to allow number of filters to grow
    '''

    eps = 1.1e-5
    concat_feat = x

    for i in range(nb_layers):
        branch = i+1
        x = conv_block(concat_feat, stage, branch, growth_rate, dropout_rate, weight_decay)
        concat_feat = merge([concat_feat, x], mode='concat', concat_axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))

        if grow_nb_filters:
            nb_filter += growth_rate

    return concat_feat, nb_filter

In [16]:
img_rows, img_cols = 128,128 # Resolution of inputs
channel = 3
num_classes = 85
batch_size = 16 
nb_epoch = 10



# Load our model
model = densenet169_model(img_rows=img_rows, img_cols=img_cols, color_type=channel, num_classes=num_classes)


/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (7, 7), strides=(2, 2), use_bias=False, name="conv1")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:107: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv2_1_x1")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:117: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv2_1_x2")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:173: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/keras/legacy/layers.py:465: Us

/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:144: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), use_bias=False, name="conv3_blk")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:107: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_1_x1")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:117: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_1_x2")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:107: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_2_x1")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:117: UserWarning: Update your `Conv2D` call to the Keras 2 API:

/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:107: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_20_x1")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:117: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_20_x2")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:107: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_21_x1")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:117: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_21_x2")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:107: UserWarning: Update your `Conv2D` call to the Keras 2 

/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:107: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv5_7_x1")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:117: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv5_7_x2")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:107: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv5_8_x1")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:117: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv5_8_x2")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:107: UserWarning: Update your `Conv2D` call to the Keras 2 API:

/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:107: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv5_27_x1")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:117: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv5_27_x2")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:107: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv5_28_x1")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:117: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv5_28_x2")`
/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:107: UserWarning: Update your `Conv2D` call to the Keras 2 

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv1_zeropadding (ZeroPadding2 (None, 134, 134, 3)  0           data[0][0]                       
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   9408        conv1_zeropadding[0][0]          
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_scal

__________________________________________________________________________________________________
conv5_15_x2_scale (Scale)       (None, 4, 4, 128)    256         conv5_15_x2_bn[0][0]             
__________________________________________________________________________________________________
relu5_15_x2 (Activation)        (None, 4, 4, 128)    0           conv5_15_x2_scale[0][0]          
__________________________________________________________________________________________________
conv5_15_x2_zeropadding (ZeroPa (None, 6, 6, 128)    0           relu5_15_x2[0][0]                
__________________________________________________________________________________________________
conv5_15_x2 (Conv2D)            (None, 4, 4, 32)     36864       conv5_15_x2_zeropadding[0][0]    
__________________________________________________________________________________________________
concat_5_15 (Merge)             (None, 4, 4, 1120)   0           concat_5_14[0][0]                
          

In [16]:
early_stops = EarlyStopping(patience=3, monitor='val_acc')
checkpointer = ModelCheckpoint(filepath='weights_densenet-169.hdf5', verbose=1, save_best_only=True)
# Start Fine-tuning

model.fit(Xtrain, ytrain,
          validation_data=(Xvalid, yvalid),
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          shuffle=True,
          verbose=1,
          callbacks=[checkpointer])

/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # This is added back by InteractiveShellApp.init_path()


Train on 10080 samples, validate on 2520 samples
Epoch 1/10
10080/10080 [==============================] - 5645s 560ms/step - loss: 0.5027 - acc: 0.7484 - val_loss: 0.3977 - val_acc: 0.8187

Epoch 00001: val_loss improved from inf to 0.39775, saving model to weights_densenet-169.hdf5
Epoch 2/10
10080/10080 [==============================] - 5472s 543ms/step - loss: 0.3517 - acc: 0.8456 - val_loss: 0.3041 - val_acc: 0.8725

Epoch 00002: val_loss improved from 0.39775 to 0.30411, saving model to weights_densenet-169.hdf5
Epoch 3/10
10080/10080 [==============================] - 5458s 541ms/step - loss: 0.2924 - acc: 0.8772 - val_loss: 0.2556 - val_acc: 0.8969

Epoch 00003: val_loss improved from 0.30411 to 0.25562, saving model to weights_densenet-169.hdf5
Epoch 4/10
10080/10080 [==============================] - 5475s 543ms/step - loss: 0.2565 - acc: 0.8959 - val_loss: 0.2235 - val_acc: 0.9123

Epoch 00004: val_loss improved from 0.25562 to 0.22353, saving model to weights_densenet-169.

# further training

In [17]:
early_stops = EarlyStopping(patience=3, monitor='val_acc')
checkpointer = ModelCheckpoint(filepath='weights_densenet-169_1.hdf5', verbose=1, save_best_only=True)
# Start Fine-tuning
model.load_weights('weights_densenet-169.hdf5')
model.fit(Xtrain, ytrain, 
          validation_data=(Xvalid, yvalid),
          batch_size=16,
          nb_epoch=15,
          shuffle=True,
          verbose=1,
          callbacks=[checkpointer])

/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # This is added back by InteractiveShellApp.init_path()


Train on 10080 samples, validate on 2520 samples
Epoch 1/15
10080/10080 [==============================] - 5826s 578ms/step - loss: 0.1440 - acc: 0.9487 - val_loss: 0.1453 - val_acc: 0.9438

Epoch 00001: val_loss improved from inf to 0.14533, saving model to weights_densenet-169_1.hdf5
Epoch 2/15
10080/10080 [==============================] - 5640s 560ms/step - loss: 0.1358 - acc: 0.9519 - val_loss: 0.1419 - val_acc: 0.9448

Epoch 00002: val_loss improved from 0.14533 to 0.14188, saving model to weights_densenet-169_1.hdf5
Epoch 3/15
10080/10080 [==============================] - 5508s 546ms/step - loss: 0.1259 - acc: 0.9566 - val_loss: 0.1370 - val_acc: 0.9469

Epoch 00003: val_loss improved from 0.14188 to 0.13705, saving model to weights_densenet-169_1.hdf5
Epoch 4/15
10080/10080 [==============================] - 5604s 556ms/step - loss: 0.1174 - acc: 0.9600 - val_loss: 0.1351 - val_acc: 0.9475

Epoch 00004: val_loss improved from 0.13705 to 0.13514, saving model to weights_densene

KeyboardInterrupt: 

In [ ]:
early_stops = EarlyStopping(patience=3, monitor='val_acc')
checkpointer = ModelCheckpoint(filepath='weights_densenet-169_2.hdf5', verbose=1, save_best_only=True)
# Start Fine-tuning
model.load_weights('weights_densenet-169_1.hdf5')
model.fit(Xtrain, ytrain, 
          validation_data=(Xvalid, yvalid),
          batch_size=16,
          nb_epoch=15,
          shuffle=True,
          verbose=1,
          callbacks=[checkpointer])

# prediction


In [18]:
model.load_weights('weights_densenet-169_1.hdf5')
pred_test = model.predict(X_test).round()
pred_test = pred_test.astype(np.int)

In [19]:
#submission
subm = pd.DataFrame()

subm['Image_name'] = test.Image_name

label_df = pd.DataFrame(data=pred_test, columns=label_cols)

subm = pd.concat([subm, label_df], axis=1)

subm.to_csv('submit_densenet_169_1.csv', index=False)